In [71]:
import redis
import pandas as pd
from datetime import datetime

db = redis.Redis(host='127.0.0.1', port=6379, db=0, charset="utf-8", decode_responses=True)
db.flushdb()

True

In [72]:
sobrante=pd.read_csv('C:/Users/Pabul/Downloads/productosTest.csv')
sobrante

,Producto,Cantidad
0,Volovan,10
1,Princesa,12
2,Megas,2
3,Paseado,17
4,Pastor,33
5,Reynas,1
6,Conos,0
7,Gaznates,9
8,Pizza,17


In [77]:

def carga_productos(restantes):
    for index, row in sobrante.iterrows():
        key= 'Productos:'+row[0]
        result= db.set(key, int(row[1]))
    return result

def get_sobrantes(producto):
    key="Productos:"+producto
    result = db.get(key)
    return int(result)

def producidos(producto, producido):
    key="Productos:"+producto
    result= db.set(key, get_sobrantes(producto)+producido)
    return result

def comprados(producto, comprado):
    key="Productos:"+producto
    result= db.set(key, get_sobrantes(producto)-comprado)
    return result

def gastosExtra(gastado, concepto, user):
    hora = datetime.now()
    key1='gastoExtra:Gasto:'
    key2='gastoExtra:Concepto:'
    key3='gastoExtra:Hora:'
    key4='gastoExtra:Usuario:'
    db.rpush(key1, gastado)
    db.rpush(key2, concepto)
    db.rpush(key3, hora)
    db.rpush(key4, user)
    return 

def gastosExtraTotales():
    gastos= db.lrange('gastoExtra:Gasto:', 0, -1)
    return sum(map(int, gastos))

def ingresosExtra(ingresado, concepto, user):
    hora = datetime.now()
    key1='ingresoExtra:Ingreso:'
    key2='ingresoExtra:Concepto:'
    key3='ingresoExtra:Hora:'
    key4='ingresoExtra:Usuario:'
    db.rpush(key1, ingresado)
    db.rpush(key2, concepto)
    db.rpush(key3, hora)
    db.rpush(key4, user)
    return

def ingresosExtraTotales():
    gastos= db.lrange('ingresoExtra:Ingreso:', 0, -1)
    return sum(map(int, gastos))

def dfIngresosExtra():
    ingdict={'Fecha': db.lrange('ingresoExtra:Hora:', 0, -1), 'Concepto':db.lrange('ingresoExtra:Concepto:', 0, -1), \
        'Ingreso': db.lrange('ingresoExtra:Ingreso:', 0, -1), 'Usuario': db.lrange('ingresoExtra:Usuario:', 0, -1)}
    ingdf=pd.DataFrame.from_dict(data=ingdict)
    return( ingdf)

def dfGastosExtra():
    gastdict= {'Fecha': db.lrange('gastoExtra:Hora:', 0, -1), 'Concepto':db.lrange('gastoExtra:Concepto:', 0, -1), \
        'Costo': db.lrange('gastoExtra:Gasto:', 0, -1), 'Usuario': db.lrange('gastoExtra:Usuario:', 0, -1)}
    gastodf=pd.DataFrame.from_dict(data=gastdict)
    return(gastodf)


carga_productos(sobrante)
print(get_sobrantes('Volovan'))
producidos('Volovan', 15)
print(get_sobrantes('Volovan'))
comprados('Volovan', 1)
print(get_sobrantes('Volovan'))

gastosExtra(1, 'Manzanas', 'Yeu')
gastosExtra(17, 'Peras', 'Pablo')

ingresosExtra(91, 'Anticipo', 'Marcela')
ingresosExtra(17, 'Prepago', 'Pablo')
ingresosExtra(94, 'Prepago', 'Yeu')

print(gastosExtraTotales())

print(db.lrange('gastoExtra:Gasto:', 0, -1))
print(db.lrange('gastoExtra:Concepto:', 0, -1))
print(db.lrange('gastoExtra:Hora:', 0, -1))
print(db.lrange('gastoExtra:Usuario:', 0, -1))

print(db.lrange('ingresoExtra:Ingreso:', 0, -1))
print(db.lrange('ingresoExtra:Concepto:', 0, -1))
print(db.lrange('ingresoExtra:Hora:', 0, -1))
print(db.lrange('ingresoExtra:Usuario:', 0, -1))

10
25
24
4425
['65', '47', '71', '35', '12', '11', '30', '83', '68', '79', '89', '48', '17', '84', '68', '28', '78', '18', '1', '24', '43', '72', '43', '51', '19', '82', '96', '8', '68', '57', '28', '32', '97', '55', '79', '4', '75', '16', '75', '10', '57', '93', '84', '15', '13', '97', '44', '17', '21', '63', '17', '70', '56', '18', '26', '60', '55', '20', '3', '50', '99', '85', '62', '86', '73', '43', '41', '0', '95', '26', '92', '86', '27', '22', '53', '9', '58', '7', '74', '66', '81', '40', '20', '6', '15', '45', '41', '34', '39', '14', '59', '8', '56', '1', '17']
['Detergente', 'Quita Grasa', 'Manzanas', 'Manzanas', 'Aceite', 'Charloa', 'Manzanas', 'Quita Grasa', 'Quita Grasa', 'Quita Grasa', 'Aceite', 'Aceite', 'Manzanas', 'Detergente', 'Aceite', 'Quita Grasa', 'Quita Grasa', 'Quita Grasa', 'Manzanas', 'Aceite', 'Detergente', 'Manzanas', 'Quita Grasa', 'Detergente', 'Aceite', 'Detergente', 'Detergente', 'Detergente', 'Detergente', 'Manzanas', 'Charloa', 'Manzanas', 'Charloa', 'Ma

In [74]:
datos=cerrarDia()
datos[0]

,Fecha,Concepto,Costo,Usuario
0,05/06/2022 14:37:21,Detergente,65,Yeu
1,05/06/2022 14:37:27,Quita Grasa,47,ProfePime
2,05/06/2022 14:37:30,Manzanas,71,Yeu
3,05/06/2022 14:37:30,Manzanas,35,Marcela
4,05/06/2022 14:37:38,Aceite,12,ProfePime
...,...,...,...,...
88,05/06/2022 14:42:08,Charloa,39,Pablo
89,05/06/2022 14:42:08,Detergente,14,ProfePime
90,05/06/2022 14:42:11,Manzanas,59,Yeu
91,05/06/2022 14:42:12,Aceite,8,Marcela


In [75]:
datos[1]

,Fecha,Concepto,Ingreso,Usuario
0,05/06/2022 14:37:19,Emergencias,7,Pablo
1,05/06/2022 14:37:20,Devoluciones,37,Yeu
2,05/06/2022 14:37:23,Emergencias,42,ProfePime
3,05/06/2022 14:37:24,Devoluciones,28,Marcela
4,05/06/2022 14:37:27,Emergencias,68,Marcela
...,...,...,...,...
102,05/06/2022 14:41:56,Prepago,70,Yeu
103,05/06/2022 14:42:01,Anticipo,67,Pablo
104,05/06/2022 14:42:04,Devoluciones,24,Yeu
105,05/06/2022 14:42:07,Anticipo,53,Yeu


In [73]:
"""import time
import numpy as np
#Generando datos
gconcept=['Manzanas', 'Aceite', 'Quita Grasa', 'Detergente', 'Charloa']
users=['Yeu', 'Pablo', 'Marcela', 'ProfePime']
ingconcept=['Anticipo', 'Devoluciones', 'Prepago', 'Emergencias']

for i in range(0,200):
    if(np.random.randint(2)==1):
        ingresosExtra(np.random.randint(100), ingconcept[np.random.randint(4)],users[np.random.randint(4)])
    else:
        gastosExtra(np.random.randint(100), gconcept[np.random.randint(5)],users[np.random.randint(4)])
    time.sleep(np.random.randint(4))
    """

In [76]:
datos[0].to_csv('C:/Users/Pabul/Desktop/TRABAJOS LCD 2022-1/BBDDNE/Proyecto_Final/CostosExtra.csv', index=False)
datos[1].to_csv('C:/Users/Pabul/Desktop/TRABAJOS LCD 2022-1/BBDDNE/Proyecto_Final/IngresosExtra.csv' , index=False)